# Finding a Competent Subset for Visualization, Simulation, or 3D Printing

This notebook contains functions and the workflow for finding a competent subset for visualization.

**Author Contact:** <br>
LinkedIn: [Çınar Turhan](https://www.linkedin.com/in/cinarturhan/) <br>
Email: [cinarturhan@utexas.edu](mailto:cinarturhan@utexas.edu)


## Table of Contents
[1. Finding a Competent Subset](#chapter1)<br>
[1.1 Required Modules](#section_1_1)<br>
[1.2. Functions](#section_1_2)<br>
[1.3. Loading the Data](#section_1_3)<br>
[1.4. Initial Visualizations](#section_1_4)<br>
[1.5. Finding the Competent Subsets](#section_1_5)<br>
[1.6. Final Visualizations](#section_1_5)<br>
[2. Comparison Between the Layered, and Non-Layered Edges for a Subset](#chapter2)<br>
___

# 1. Finding a Competent Subset <a class="anchor" id="chapter1"></a>

## 1.1. Required Modules <a class="anchor" id="section_1_1"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
import pyvista as pv
pv.start_xvfb()
pv.set_jupyter_backend('static')
import cc3d
import tifffile

import warnings
warnings.filterwarnings('ignore')

from copy import deepcopy
import glob
import scipy
import os

## Functions <a class="anchor" id="section_1_2"></a>

1. **find_porosity_visualization_subset**: Finds the competent visualization subset.
2. **add_layer**: Adds a layer of pores or solids around the sample to distinguish the pores in visualization, create solid boundaries for simulation, or prepare a model for 3D printing.
3. **plot_sample**: Visualizing the samples.

In [ ]:
def find_porosity_visualization_subset(data, cube_size = 100, batch=100, pore_class = 0):
    '''
    Finds the best cubic subset for visaulizing the segmented dataset.
    
    data: 3D numpy array, vector class from DPM Tools, Image class from DPM Tools.
    cube_size: Size of the visalization cube, default is 100 (100x100x100). 
    batch: Batch over which to calculate the stats, default is 100.
    
    '''
    
    if str(type(data))=="<class 'dpm_tools.io.read_data.Vector'>":
        scalar_data = deepcopy(data.image)
    elif str(type(data))=="<class 'dpm_tools.io.read_data.Image'>":
        scalar_data = deepcopy(data)
    else:
        scalar_data = deepcopy(data)

    if pore_class != 1:
        scalar_data[scalar_data==pore_class]=199
        scalar_data[scalar_data!=199]=pore_class
        scalar_data[scalar_data==199]=1

    size = scalar_data.shape[0]*scalar_data.shape[1]*scalar_data.shape[2]
    porosity = (scalar_data==1).sum()/size

    sample_size = cube_size

    # Inner cube increment
    inc = sample_size-int(sample_size*0.5)

    # One dimension of the given vector sample cube.
    max_dim = len(scalar_data)    

    batch_for_stats = max_dim-sample_size # Max possible batch number

    # Or overwrite:
    batch_for_stats = batch

    stats_array=np.zeros(shape=(5,batch_for_stats))

    i=0
    while i<batch_for_stats:
        mini = np.random.randint(low=0, high=max_dim-sample_size)
        maxi = mini+sample_size

        scalar_boot = scalar_data[mini:maxi,mini:maxi,mini:maxi]
        scalar_boot_inner = scalar_data[mini+inc:maxi-inc,mini+inc:maxi-inc,mini+inc:maxi-inc]

        labels_out_outside, N = cc3d.largest_k(
            scalar_boot, k=1, 
            connectivity=26, delta=0,
            return_N=True,
        )

        index_outside,counts_outside = np.unique(labels_out_outside,return_counts=True)
        counts_outside_sum = np.sum(counts_outside[1:])

        labels_out_inside, N = cc3d.largest_k(
            scalar_boot_inner, k=1, 
            connectivity=26, delta=0,
            return_N=True,
        )

        index_inside,counts_inside = np.unique(labels_out_inside,return_counts=True)
        counts_inside_sum = np.sum(counts_inside[1:])

        porosity_selected = (scalar_boot==1).sum()/sample_size**3

        if (porosity_selected<=porosity*1.2)&(porosity_selected>=porosity*0.8):
            stats_array[0,i] = counts_outside_sum
            stats_array[1,i] = counts_inside_sum     
            stats_array[2,i] = porosity_selected   
            stats_array[3,i] = mini
            stats_array[4,i] = scipy.stats.hmean([stats_array[0,i],
                                                  stats_array[1,i]])
            i+=1

        else:
            continue


    best_index = np.argmax(stats_array[4,:])
    best_subset_range = int(stats_array[3,best_index])

    print(f'Original Porosity: {round(porosity*100,2)} %\n' +
          f'Subset Porosity: {round(stats_array[2,best_index]*100,2)} %\n' +
          f'Competent Subset: [{best_subset_range}:{best_subset_range+cube_size},' +
          f'{best_subset_range}:{best_subset_range+cube_size},{best_subset_range}:{best_subset_range+cube_size}]')
    
    best_subset_range = (int(best_subset_range),int(best_subset_range+cube_size))
    
    return best_subset_range, stats_array

def add_layer(scalar_data, subset_range, trim_pixels_from_sides = 4, insert_phase = 'pore', pore_class = 0, solid_class = 1):
    '''
    Sophisticated way of doing numpy.pad()
    
    Takes the whole data, and the subset_range that is to be visualized. Trims the pixels from the sides,
    based on the keyword trim_pixels_from_each_sides. Replaces those with pore or solid phase based on insert_phase keyword.
    By default, pore_class is 0, and the solid_class is 1. If you have different values, set those keywords to those numbers.
    scalar_data = 3D numpy array.
    subset_range = Range of interest/Volume of interest.
    trim_pixels_from_each_sides = Thickness of the layer to be added.
    insert_phase = 'solid' or 'pore'. Phase to be inserted.
    pore_class = Value of the pores in the dataset.
    solid_class = Value of the solids in the dataset.
    '''
    mini = subset_range[0]
    maxi = subset_range[1]
    
    scalar_data = deepcopy(scalar_data)
    
    if insert_phase == 'pore':
        scalar_data[mini:mini + trim_pixels_from_sides, :, :] = pore_class
        scalar_data[:, mini:mini + trim_pixels_from_sides, :] = pore_class
        scalar_data[:, :, mini:mini + trim_pixels_from_sides] = pore_class
        scalar_data[maxi - trim_pixels_from_sides:maxi, :, :] = pore_class
        scalar_data[:, maxi - trim_pixels_from_sides:maxi, :] = pore_class
        scalar_data[:, :, maxi - trim_pixels_from_sides:maxi] = pore_class
    elif insert_phase == 'solid':
        scalar_data[mini:mini + trim_pixels_from_sides, :, :] = solid_class
        scalar_data[:, mini:mini + trim_pixels_from_sides, :] = solid_class
        scalar_data[:, :, mini:mini + trim_pixels_from_sides] = solid_class
        scalar_data[maxi - trim_pixels_from_sides:maxi, :, :] = solid_class
        scalar_data[:, maxi - trim_pixels_from_sides:maxi, :] = solid_class
        scalar_data[:, :, maxi - trim_pixels_from_sides:maxi] = solid_class
    else:
        raise Exception('Please enter a valid class: \'solid\' or \'pore\' only')
    
    sample = scalar_data[mini:maxi,mini:maxi,mini:maxi]
        
    return sample


def plot_sample(sample, subset=True, subset_range = (0,128)):
    '''
    Plots the isosurfaces between pores and solids. 
    subset: If true, assign the subset_range you want to plot. If false, it plots the whole sample.
    subset_range: If subset is set to 'True', assign the range here.
    '''
    pv.set_jupyter_backend('static')
    # pv.start_xvfb()
    plotter_obj = pv.Plotter(lighting='three lights')#notebook=True, off_screen=False, lighting='three lights')

    # Set background colors
    plotter_obj.set_background(color='w')
#     plotter_obj.add_light(pv.Light(light_type='headlight'))

    # Set font colors and sizes
    pv.global_theme.font.color = 'black'
    pv.global_theme.font.size = 18
    pv.global_theme.font.label_size = 14

    
    if subset:
        mini = subset_range[0]
        maxi = subset_range[1]
        sample = sample[mini:maxi,mini:maxi,mini:maxi]
    
    pv_image_obj = pv.wrap(sample)
    contours = pv_image_obj.contour(isosurfaces=[0.5])
    plotter_obj.add_mesh(contours, opacity=1, color=(200 / 255, 181 / 255, 152 / 255), diffuse=0.75, ambient=0.15)
    
    plotter_obj.show()

## Loading the Data <a class="anchor" id="section_1_3"></a>

In these datasets, pores are set to 1, and the solids are set to 0.

In [ ]:
img_list = glob.glob('../Data/*.ubc')
img_list

In [ ]:
# Reading Images
castlegate, sandpack, gambier, beadpack = [np.fromfile(img,dtype='uint8') for img in img_list]

# Reshaping Images
gambier = gambier.reshape((512,512,512))
castlegate = castlegate.reshape((512,512,512))
beadpack = beadpack.reshape((512,512,512))
sandpack = sandpack.reshape((512,512,512))

# Iverting the binary arrays (switching void and solid phases)
gambier = np.invert(gambier==True).astype('uint8')
castlegate = np.invert(castlegate==True).astype('uint8')
beadpack = np.invert(beadpack==True).astype('uint8')
sandpack = np.invert(sandpack==True).astype('uint8')

fig, ax = plt.subplots(nrows=2, ncols=2,figsize=(8,8))
ax = ax.flatten()

skimage.io.imshow(sandpack[0,:,:], cmap='gray', ax=ax[0])
ax[0].set_title('Sand Pack',fontsize=14)

skimage.io.imshow(gambier[0,:,:], cmap='gray', ax=ax[1])
ax[1].set_title('Gambier Limestone',fontsize=14)

skimage.io.imshow(beadpack[0,:,:], cmap='gray', ax=ax[2])
ax[2].set_title('Glass Bead Pack',fontsize=14)

skimage.io.imshow(castlegate[0,:,:], cmap='gray', ax=ax[3])
ax[3].set_title('Castlegate Sandstone',fontsize=14)

plt.show()

## Initial Visualizations <a class="anchor" id="section_1_4"></a>

In [ ]:
# subset_range = (128,256)
sandpack_vis = add_layer(sandpack, subset_range = (128,256), trim_pixels_from_sides = 1, insert_phase = 'pore', pore_class = 1, solid_class = 0)
gambier_vis = add_layer(gambier, subset_range = (128,256), trim_pixels_from_sides = 1, insert_phase = 'pore', pore_class = 1, solid_class = 0)
beadpack_vis = add_layer(beadpack, subset_range = (128,256), trim_pixels_from_sides = 1, insert_phase = 'pore', pore_class = 1, solid_class = 0)
castlegate_vis = add_layer(castlegate, subset_range = (128,256), trim_pixels_from_sides = 1, insert_phase = 'pore', pore_class = 1, solid_class = 0)

In [ ]:
plot_sample(sandpack_vis, subset=False)

In [ ]:
plot_sample(gambier_vis, subset=False)

In [ ]:
plot_sample(beadpack_vis, subset=True, subset_range = (0,128))

In [ ]:
plot_sample(castlegate_vis, subset=True, subset_range = (0,128))

In [ ]:
# Porosity of the selected subset of Gambier Limestone
segmented_Gambier_Subset_128 = gambier[128:256, 128:256, 128:256]
porosity = np.sum(segmented_Gambier_Subset_128 == 1)/(128**3)*100
print(round(porosity, 1), '%')

In [ ]:
# Original porosity of the limestone
print(round(np.sum(gambier == 1)/(512**3)*100, 1), '%')

## Finding the Competent Subsets <a class="anchor" id="section_1_4"></a>

In [ ]:
# subset1, _ = find_porosity_visualization_subset(sandpack, cube_size=128, batch=100, pore_class = 1)

In [ ]:
subset2, _ = find_porosity_visualization_subset(gambier, cube_size=128, batch=100, pore_class = 1)

In [ ]:
subset2

In [ ]:
# subset3, _ = find_porosity_visualization_subset(beadpack, cube_size=128, batch=100, pore_class = 1)

In [ ]:
# subset4, _ = find_porosity_visualization_subset(castlegate, cube_size=128, batch=100, pore_class = 1)

## Final Visualizations <a class="anchor" id="section_1_5"></a>

In [ ]:
# segmented_lrc32_vis = add_layer(sandpack, subset_range = subset1, trim_pixels_from_sides = 2, insert_phase = 'pore', pore_class = 1, solid_class = 0)
segmented_Gambier_vis = add_layer(gambier, subset_range = subset2, trim_pixels_from_sides = 2, insert_phase = 'pore', pore_class = 1, solid_class = 0)
# segmented_bead_pack_512_vis = add_layer(beadpack, subset_range = subset3, trim_pixels_from_sides = 2, insert_phase = 'pore', pore_class = 1, solid_class = 0)
# segmented_castle_512_vis = add_layer(castlegate, subset_range = subset4, trim_pixels_from_sides = 2, insert_phase = 'pore', pore_class = 1, solid_class = 0)

# plot_sample(segmented_lrc32_vis, subset=False)
plot_sample(segmented_Gambier_vis, subset=False)
# plot_sample(segmented_bead_pack_512_vis, subset=False)
# plot_sample(segmented_castle_512_vis, subset=False)

## Comparison Between the Layered, and Non-Layered Edges for a Subset <a class="anchor" id="chapter2"></a>

In [ ]:
segmented_Gambier_vis = add_layer(gambier, subset_range = subset2, 
                                  trim_pixels_from_sides = 2, 
                                  insert_phase = 'pore', 
                                  pore_class = 1, 
                                  solid_class = 0)

plot_sample(segmented_Gambier_vis,subset=False)
plot_sample(gambier,subset=True, subset_range=subset2)

____

Author: [Çınar Turhan](https://www.linkedin.com/in/cinarturhan/)